# Case Study: Growth

## Introduction

Company A is a popular photo editing product with millions of active users. The Growth Unit is responsible for expanding the user base, increasing user engagement, and retaining users. 
They are interested in understanding **user behavior and engagement** with its product. Below, a dataset containing user activity data was provided. Your task is to analyze this data and provide **actionable insights** and recommendations to **improve user growth and engagement**. 
In this task, you will be working with datasets that contain event logs and app open data for a specified period.

#### Part 1
1- Write a SQL query to derive the first session funnel per platform from the provided dataset using all available events. The first funnel step is the user’s first app open.
2- Calculate the WAU Growth Accounting (New, Retained, Resurrected, Churn users) from November 13th to November 19th
3- Write a SQL query to calculate the user’s daily retention rate for a 7-day period. Calculate both Sticky and Cohorted Retention

#### Part 2 
1- Please perform an analysis using the dataset provided. The goal is to find insights and suggest action(s) to stakeholders through your findings. Use Python, R, or a similar programming language to generate an analysis that could be shared/reviewed by other analysts.

Expected output: A memo in a form of Google doc that describes the finding and outlines action items. Additional attachments (spreadsheets, Jupyter notebooks, etc.) only if it is necessary

## Data Understanding and cleaning

#### App Open 
______________________________________________________

| Column        | Data Type         | Description                               | Unique Key|
| ----------- | ----------- | ----------- | ----------- |
| timestamp     |TIMESTAMP          |Timestamp when the app was opened          |No|
| device_skey   |STRING             |Unique identifier for the device           |No|
| session_skey  |STRING             |Unique identifier for the session          |No|
| user_skey     |STRING             |Unique identifier for the users            |No|
| is_first_app_open |BOOLEAN        |Indicator of whether it's the first time app open for the user |No|
| platform      |STRING             |Platform on which the app was opened (e.g., android, apple) |No|
| country_code  |STRING             |Country from which the app was opened (e.g., us, de) |No|

In [ ]:
apps_open = _deepnote_execute_sql('SELECT \n    timestamp::TIMESTAMP_S AS date_time\n    , device_skey::VARCHAR AS device_skey\n    , session_skey::VARCHAR AS session_skey\n    , user_skey::VARCHAR AS user_skey\n    , is_first_app_open::BOOLEAN AS is_first_app_open\n    , platform::VARCHAR AS platform\n    , country_code::VARCHAR AS country_code\nFROM \'ps_challanges/growth_case_study/Datasets/App Opens.csv\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
apps_open

,date_time,device_skey,session_skey,user_skey,is_first_app_open,platform,country_code
0,2023-11-08 02:07:19,7891415701568881512,2011583431778737851,-8717167265529084706,False,android,br
1,2023-11-08 06:26:35,2841158506866498595,-710585864896703953,3858142552250413010,False,android,vn
2,2023-11-08 11:00:09,8319180276841915965,8851130170214692120,-6901249382849255824,False,apple,vn
3,2023-11-08 17:03:55,-7044717482868540199,2562809385083234043,42,False,apple,vn
4,2023-11-08 10:09:35,8330275025360404105,6269721242185127990,-8100364697395241128,False,android,de
...,...,...,...,...,...,...,...
196085,2023-11-15 11:47:15,-1985881793666309748,1078423291596019483,3098934470375878848,False,android,br
196086,2023-11-15 15:24:53,2134527128238454256,-3747564580958942091,-8099055897267782491,False,apple,br
196087,2023-11-15 15:31:40,-2987017123302243105,-6832478710689874856,-8250782251817560378,False,android,vn
196088,2023-11-15 20:41:23,1438462127647457196,-5384106018072454101,-46678441358607097,False,android,br


#### Events
______________________________________________________


| Column| Data Type| Description| Unique Key|
| ----------- | ----------- | ----------- | ----------- |
event_name |STRING |Type of event occurred |No
timestamp |TIMESTAMP |Timestamp when the event occurred |No
event_skey |STRING |Unique identifier for the event |Yes
device_skey |STRING |Unique identifier for the device |No
user_skey |STRING |Unique identifier for the user |No
session_skey |STRING |Unique identifier for the session |No
platform |STRING |Platform on which the event occurred (e.g., android, apple) |No
source |STRING |Source screen from which the event was triggered |No
country_code |STRING |Country from which the app was opened (e.g., us, de) | No

In [ ]:
events = _deepnote_execute_sql('SELECT \n    event_name::STRING AS event_name\n    , epoch_ms(timestamp)::TIMESTAMP_S AS date_time\n    , event_skey::STRING AS event_skey\n    , device_skey::STRING AS device_skey\n    , user_skey::STRING AS user_skey\n    , session_skey::STRING AS session_skey\n    , platform::STRING AS platform\n    , source::STRING AS source\n    , country_code::STRING AS country_code\nFROM \'ps_challanges/growth_case_study/Datasets/Events.csv\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
events

,event_name,date_time,event_skey,device_skey,user_skey,session_skey,platform,source,country_code
0,registration_open,2023-11-11 03:32:05,-927509160255567445,-724186991312688217,null,5342905048311674142,apple,app_start,vn
1,registration_open,2023-11-11 03:16:02,6726005894254876970,167579040429311423,null,1033335939576521366,android,app_start,vn
2,registration_open,2023-11-10 15:18:40,-2921948449119298022,-753449636207296314,null,7698330632377898188,apple,app_start,vn
3,registration_open,2023-11-10 15:46:26,920368455108193822,6151555484025284746,null,1455590834684669000,android,user_profile,br
4,registration_open,2023-11-10 15:46:35,4301569710999595535,6151555484025284746,null,1455590834684669000,android,user_profile,br
...,...,...,...,...,...,...,...,...,...
762098,object_export,2023-11-06 03:19:28,8762455078106490544,-218442561903040844,-7394739150698595004,4931081474184387585,apple,editor_screen,vn
762099,object_export,2023-11-06 03:19:33,1236197224848669172,-8953068508614011099,-5616148740068329164,1939940093157332532,apple,editor_add_objects,vn
762100,object_export,2023-11-06 00:40:15,-1128268909705829967,7622198276030932501,-4476277103857062859,-914698409805777184,apple,share_screen,br
762101,object_export,2023-11-06 03:46:08,-5041445763328466000,-2972284481220770541,5642301744160861251,-2767914012090684930,android,editor_add_objects,eg


In [ ]:
DeepnoteChart(events, """{"mark":{"type":"bar","tooltip":true},"width":"container","config":{"customFormatTypes":true},"height":"container","$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{"x":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"y":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear"},"format":{"type":"default","decimals":null}}}}""")

#### Visitors
______________________________________________________

| Column| Data Type| Description| Unique Key|
| ----------- | ----------- | ----------- | ----------- |
|Date |STRING| Date of visit | yes
Store Visitors. Source: search | BIGINT |Number of store visitors from the source search |No
Store Visitors. Source: explore | BIGINT |Number of store visitors from the source explore |No
Store Visitors. Source: referrals | BIGINT |Number of store visitors from the source referrals |No

In [ ]:
visitors = _deepnote_execute_sql('SELECT \n    Date::TIMESTAMP_S AS date_time\n    , "Store Visitors. Source: search"::BIGINT AS store_visitors_source_search\n    , "Store Visitors. Source: explore"::BIGINT AS store_visitors_source_explore\n    , "Store Visitors. Source: referrals"::BIGINT AS store_visitors_source_referrals\nFROM \'ps_challanges/growth_case_study/Datasets/store_visitors.csv\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
visitors

,date_time,store_visitors_source_search,store_visitors_source_explore,store_visitors_source_referrals
0,2023-11-01,6,6,5
1,2023-11-02,7,8,7
2,2023-11-03,3,3,2
3,2023-11-04,286,3,3
4,2023-11-05,7,9,6
5,2023-11-06,2,4,2
6,2023-11-07,4,7,4
7,2023-11-08,1,3,2
8,2023-11-09,6,12,6
9,2023-11-10,8,15,7


## Part 1

> 1- Write an SQL query to derive the first session funnel per platform from the provided dataset using all available events. The first funnel step is the user’s first app open.

In [ ]:
first_sessio_funnel = _deepnote_execute_sql('WITH base AS (\n    SELECT events.*\n    , RANK() OVER (PARTITION BY events.user_skey ORDER BY events.date_time DESC) AS rank_event\n    FROM events\n    INNER JOIN apps_open USING (session_skey) \n    WHERE apps_open.is_first_app_open = TRUE\n)\n\nSELECT \n    platform\n    , event_name\n    , count(*) AS cnt\nFROM base\nWHERE rank_event = 1\nGROUP BY 1, 2\nORDER BY cnt desc', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
first_sessio_funnel

,platform,event_name,cnt
0,android,subscription_offer_open,5
1,android,editor_open,3
2,android,object_export,2
3,apple,object_export,2
4,apple,editor_open,1
5,android,create_flow_open,1


> 2. Calculate the WAU Growth Accounting (New, Retained, Resurrected, Churn users) from November 13th to November 19th

1. New Users: Users who start using the application during the current week.
2. Retained Users: Users who also used the application in the current week and the week before.
3. Resurrected Users: Users who used the app in the current week, but not the previous week, and at some point prior to the previous week.
4. Churned Users: Users who used the app in the previous week, but not the current week.

Since the given dataset does not have a user creation date, we can't calculate the exact number of New Users. We don't have enough information about whether a user is newly registered or not.
However, we can calculate the retained, resurrected, and churned users. But for this, we need the user activity data for at least two weeks before the start date (November 13th).

In [ ]:
# Below is an illustrative example of how you can perform this calculation, given that we have the necessary data.

# Let's assume we have a DataFrame 'df' where we have the following columns:
# 'user_skey' - Unique identifier for the user.
# 'week_year' - The week of the year when the user had an activity (calculated from 'date_time').

'''
# Define the current and previous week of the year
current_week = 46 # November 13-19 falls in the 46th week of the year
previous_week = current_week - 1
week_before_previous = current_week - 2

# Compute user activity for the relevant weeks
user_by_week = df.groupby(['user_skey', 'week_year']).size().reset_index()

# Hypothetical user categories based on activity
new_users = user_by_week[user_by_week['week_year'] == current_week]
retained_users = user_by_week[(user_by_week['week_year'] == current_week) & (user_by_week['week_year'].shift(-1) == previous_week)]
resurrected_users = user_by_week[(user_by_week['week_year'] == current_week) & (user_by_week['week_year'].shift(-1) == week_before_previous)]
churned_users = user_by_week[(user_by_week['week_year'] == previous_week) & (user_by_week['week_year'].shift(-1) != current_week)]

# Count of each user type
num_new_users = len(new_users)
num_retained_users = len(retained_users)
num_resurrected_users = len(resurrected_users)
num_churned_users = len(churned_users)
'''
# Note: The execution of this code has been disabled by putting it as a comment, this is because there's no actual
# dataframe named 'df' and no week_year features. This is a high level explanation of how you could implement the wau
# growth accounting with python, provided you have the correct data.

"\n# Define the current and previous week of the year\ncurrent_week = 46 # November 13-19 falls in the 46th week of the year\nprevious_week = current_week - 1\nweek_before_previous = current_week - 2\n\n# Compute user activity for the relevant weeks\nuser_by_week = df.groupby(['user_skey', 'week_year']).size().reset_index()\n\n# Hypothetical user categories based on activity\nnew_users = user_by_week[user_by_week['week_year'] == current_week]\nretained_users = user_by_week[(user_by_week['week_year'] == current_week) & (user_by_week['week_year'].shift(-1) == previous_week)]\nresurrected_users = user_by_week[(user_by_week['week_year'] == current_week) & (user_by_week['week_year'].shift(-1) == week_before_previous)]\nchurned_users = user_by_week[(user_by_week['week_year'] == previous_week) & (user_by_week['week_year'].shift(-1) != current_week)]\n\n# Count of each user type\nnum_new_users = len(new_users)\nnum_retained_users = len(retained_users)\nnum_resurrected_users = len(resurrected_u

As I've mentioned before, since we don't have the dates that the users first registered or the exact weekly activity per user, it's impossible to calculate the Weekly Active User Growth Accounting directly with SQL queries on the specified dataset.

To calculate the Weekly Active User (WAU) Growth Accounting - namely the number of new, retained, and resurrected, and churned users - we need at least a few weeks of event data (or any sign of user activity), along with the date of the users' first activity (to identify new users). Once we have this data, we can perform calculations to figure out whether users are new, retained, resurrected or churned.

This process cannot be completed with a single SQL query on the provided dataset because:

1. New Users: The app open dataset lacks information regarding when it's the user's first ever app open. So, we can't confidently determine when the user is "new".
2. Retained Users: If the user has any activity in both the current week and the prior week, they are considered "Retained". However, our dataset includes only one week of data, so we can't calculate retained users.
3. Resurrected Users: When a user has no activity in the previous week, but some in the current week and in some week prior to the previous week, they are considered "Resurrected". Here, week-level data for three weeks is required, which we don't have.
4. Churned Users: When a user has activity in the previous week but not in the current week, we consider them to have "Churned". Again, we don't have activity recorded on a weekly basis so we can't calculate churned users.

If we had this data available, we would have to extract it for different periods (current week, previous week, and week prior to that) and make calculations accordingly, as demonstrated in the Python code example.

This work would be typically achieved by using SQL's window functions and self joins on the data table considering date intervals, but due to the lack of necessary data in this context, a SQL query isn't provided. 

If we had this kind of data, a sample SQL pseudocode might look somewhat like this:

```sql
-- an example pseudocode for SQL

-- first create temporary tables for each week
CREATE TEMPORARY TABLE current_week AS
SELECT user_skey
FROM events 
WHERE date_time BETWEEN '2023-11-13' AND '2023-11-19';

CREATE TEMPORARY TABLE previous_week AS
SELECT user_skey
FROM events
WHERE date_time BETWEEN '2023-11-06' AND '2023-11-12';

CREATE TEMPORARY TABLE week_before_previous AS
SELECT user_skey
FROM events
WHERE date_time < '2023-11-06';

-- then calculate user types
SELECT 
    COUNT(DISTINCT t1.user_skey) AS new_users,
    COUNT(DISTINCT t2.user_skey) AS retained_users,
    COUNT(DISTINCT t3.user_skey) AS resurrected_users
FROM 
    current_week t1
    LEFT JOIN previous_week t2 ON t1.user_skey = t2.user_skey
    LEFT JOIN week_before_previous t3 ON t1.user_skey = t3.user_skey
WHERE -- conditions defining new, retained, and resurrected users
;

-- Churned users query
SELECT 
    COUNT(DISTINCT t1.user_skey) AS churned_users
FROM 
    previous_week t1
    LEFT JOIN current_week t2 ON t1.user_skey = t2.user_skey
WHERE -- conditions defining churned users
;
```

Remember to replace the appropriate date timestamps according to your databases, and also make sure to have ahead the provided date intervals to compute such metrics, due to its nature on comparing consecutive periods (weeks, in this case).

Remember this is a pseudo code, so this is constructed just to give an overview of how to achieve this operation with SQL given the necessary data for it. You would only need to replace the conditions to classify each user into new, retained, resurrected, or churned. 

Also, keep in mind that this case requires recordkeeping for the timestamp of each user's activity to help track precisely when they have active events. This is a simplified case - real world scenarios will probably require additional consideration, for instance accounting for country, or platform, among others.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ea0dd5ec-7a99-4815-85b5-15ffe4204711' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>